In [ ]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 60.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import os
import random

from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
CFG = {
    'NBITS':2048,
    'SEED':42,
}

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(CFG['SEED'])  # Seed 고정

In [ ]:
def smiles_to_fingerprint(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=CFG['NBITS'])
        return np.array(fp)
    else:
        return np.zeros((CFG['NBITS'],))

In [ ]:
chembl_data = pd.read_csv('/content/drive/MyDrive/dacon_challenge/train.csv')  # 예시 파일 이름
chembl_data.head()

,Molecule ChEMBL ID,Standard Type,Standard Relation,Standard Value,Standard Units,pChEMBL Value,Assay ChEMBL ID,Target ChEMBL ID,Target Name,Target Organism,Target Type,Document ChEMBL ID,IC50_nM,pIC50,Smiles
0,CHEMBL4443947,IC50,'=',0.022,nM,10.66,CHEMBL4361896,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,SINGLE PROTEIN,CHEMBL4359855,0.022,10.66,CN[C@@H](C)C(=O)N[C@H](C(=O)N1C[C@@H](NC(=O)CC...
1,CHEMBL4556091,IC50,'=',0.026,nM,10.59,CHEMBL4345131,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,SINGLE PROTEIN,CHEMBL4342485,0.026,10.59,CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...
2,CHEMBL4566431,IC50,'=',0.078,nM,10.11,CHEMBL4345131,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,SINGLE PROTEIN,CHEMBL4342485,0.078,10.11,CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...
3,CHEMBL4545898,IC50,'=',0.081,nM,10.09,CHEMBL4345131,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,SINGLE PROTEIN,CHEMBL4342485,0.081,10.09,CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...
4,CHEMBL4448950,IC50,'=',0.099,nM,10.00,CHEMBL4361896,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,SINGLE PROTEIN,CHEMBL4359855,0.099,10.00,COc1cc2c(OC[C@@H]3CCC(=O)N3)ncc(C#CCCCCCCCCCCC...


In [ ]:
train = chembl_data[['Smiles', 'pIC50']]
train['Fingerprint'] = train['Smiles'].apply(smiles_to_fingerprint)

train_x = np.stack(train['Fingerprint'].values)
train_y = train['pIC50'].values

[00:26:42] DEPRECATION WARNING: please use MorganGenerator
[00:26:42] DEPRECATION WARNING: please use MorganGenerator
[00:26:42] DEPRECATION WARNING: please use MorganGenerator
[00:26:42] DEPRECATION WARNING: please use MorganGenerator
[00:26:42] DEPRECATION WARNING: please use MorganGenerator
[00:26:42] DEPRECATION WARNING: please use MorganGenerator
[00:26:42] DEPRECATION WARNING: please use MorganGenerator
[00:26:42] DEPRECATION WARNING: please use MorganGenerator
[00:26:42] DEPRECATION WARNING: please use MorganGenerator
[00:26:42] DEPRECATION WARNING: please use MorganGenerator
[00:26:42] DEPRECATION WARNING: please use MorganGenerator
[00:26:42] DEPRECATION WARNING: please use MorganGenerator
[00:26:42] DEPRECATION WARNING: please use MorganGenerator
[00:26:42] DEPRECATION WARNING: please use MorganGenerator
[00:26:42] DEPRECATION WARNING: please use MorganGenerator
[00:26:42] DEPRECATION WARNING: please use MorganGenerator
[00:26:42] DEPRECATION WARNING: please use MorganGenerat

In [ ]:
# 학습 및 검증 데이터 분리
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.3, random_state=42)

# 랜덤 포레스트 모델 학습
model = RandomForestRegressor(random_state=CFG['SEED'])
model.fit(train_x, train_y)

RandomForestRegressor(random_state=42)

In [ ]:
def pIC50_to_IC50(pic50_values):
    """Convert pIC50 values to IC50 (nM)."""
    return 10 ** (9 - pic50_values)

In [ ]:
# Validation 데이터로부터의 학습 모델 평가
val_y_pred = model.predict(val_x)
mse = mean_squared_error(pIC50_to_IC50(val_y), pIC50_to_IC50(val_y_pred))
rmse = np.sqrt(mse)

print(f'RMSE: {rmse}')

RMSE: 2169.5781089857264


In [ ]:
test = pd.read_csv('/content/drive/MyDrive/dacon_challenge/test.csv')
test['Fingerprint'] = test['Smiles'].apply(smiles_to_fingerprint)

[00:27:49] DEPRECATION WARNING: please use MorganGenerator
[00:27:49] DEPRECATION WARNING: please use MorganGenerator
[00:27:49] DEPRECATION WARNING: please use MorganGenerator
[00:27:49] DEPRECATION WARNING: please use MorganGenerator
[00:27:49] DEPRECATION WARNING: please use MorganGenerator
[00:27:49] DEPRECATION WARNING: please use MorganGenerator
[00:27:49] DEPRECATION WARNING: please use MorganGenerator
[00:27:49] DEPRECATION WARNING: please use MorganGenerator
[00:27:49] DEPRECATION WARNING: please use MorganGenerator
[00:27:49] DEPRECATION WARNING: please use MorganGenerator
[00:27:49] DEPRECATION WARNING: please use MorganGenerator
[00:27:49] DEPRECATION WARNING: please use MorganGenerator
[00:27:49] DEPRECATION WARNING: please use MorganGenerator
[00:27:49] DEPRECATION WARNING: please use MorganGenerator
[00:27:49] DEPRECATION WARNING: please use MorganGenerator
[00:27:49] DEPRECATION WARNING: please use MorganGenerator
[00:27:49] DEPRECATION WARNING: please use MorganGenerat

In [ ]:
test_x = np.stack(test['Fingerprint'].values)

test_y_pred = model.predict(test_x)

submit = pd.read_csv('/content/drive/MyDrive/dacon_challenge/sample_submission.csv')
submit['IC50_nM'] = pIC50_to_IC50(test_y_pred)
submit.head()

submit.to_csv('./baseline_submit.csv', index=False)